In [1]:
from langchain.chat_models import ChatOpenAI
from langchain.experimental.plan_and_execute import PlanAndExecute, load_agent_executor, load_chat_planner

/Users/ming8525/miniforge3/envs/langchain/lib/python3.9/site-packages/deeplake/util/check_latest_version.py:32: UserWarning: A newer version of deeplake (3.6.7) is available. It's recommended that you update to the latest version using `pip install -U deeplake`.
  warnings.warn(


In [2]:
from pydantic import BaseModel, Field
from langchain.agents.tools import Tool


def company_query_func(args):
    name = (args if isinstance(args, str) else args.get('name')).lower()
    members = {
        "james": "James is a member of esri company",
        "jim": "Jim is a member of google company",
        "jimmy": "Jimmy is a member of openai company",
        "jimothy": "Jimothy is a member facebook company",
    }
    return f"{members[name]}"


class CompanyQuery(BaseModel):
    name: str = Field()


company_query_tool = Tool.from_function(
    name="CompanyQuery",
    func=company_query_func,
    description="useful for when you want to know someone's company",
    args_schema=CompanyQuery
)


def company_info_func(args):
    company = (args if isinstance(args, str) else args.get('company')).lower()
    companys = {
        "esri": "esri is a geo related company",
        "google": "google is a search engine related company",
    }
    return f"{companys[company]}"


class CompanyInfo(BaseModel):
    company: str = Field()


company_info_tool = Tool.from_function(
    name="GetCompanyInfo",
    func=company_info_func,
    description="useful for when you want to know a company's info",
    args_schema=CompanyInfo
)

In [3]:
input = 'James belongs to which company? Introduce the company\'s information'
tools = [company_query_tool, company_info_tool]

In [4]:
model = ChatOpenAI(temperature=0)

planner = load_chat_planner(model)
executor = load_agent_executor(model, tools, verbose=False)
agent = PlanAndExecute(planner=planner, executor=executor, verbose=False)

In [5]:

agent.run(input)

Plan:
1. Gather information about James.
2. Search for James' employment information.
3. Identify the company James belongs to.
4. Retrieve the company's information.
5. Introduce the company's information.
6. Given the above steps taken, respond to the user's original question.


*****

Step: Gather information about James.


Response: I have found that James is a member of the Esri company.
*****

Step: Search for James' employment information.


Response: Esri is a company that specializes in geographic information system (GIS) software and technology. They provide a wide range of products and services for mapping, spatial analytics, and location intelligence. Some of their popular products include ArcGIS, ArcGIS Online, and ArcGIS Pro.
*****

Step: Identify the company James belongs to.


Response: James belongs to the Esri company.
*****

Step: Retrieve the company's information.


Response: The company Esri specializes in geographic information system (GIS) software and technolog

'James belongs to the Esri company.'